# LATAM Challenge Data Engineer

## 1. Descripción del caso

El ejercicio de estudio del desafio corresponde al análisis de datos provenientes desde Twitter, actualmente 'X', en donde se busca rescatar información estadistica de los tweets relacionados a las protestas en India de sus granjeros. Estos análisis estan enfocados en dos grandes metricas, la utilización de memoria y tiempo de cada uno de las funciones que buscan entregar respuesta a las preguntas planteadas.

## 2. Supuestos y consideraciones

Se realizo una revisión inicial a los datos proporcionados, revisando la existencia de tweets duplicados, o si existen más de una mensión a la misma persona en el mismo tweets de la siguiente manera:


In [32]:
import polars as pl
import jsonlines

data = []
file_path = "../data/farmers-protest-tweets-2021-2-4.json"
with jsonlines.open(file_path) as reader:
    for obj in reader:
        users = []
        users_dedup = []
        mentioned_users = obj.get('mentionedUsers', {})
        if mentioned_users:
            users.extend([item['username'] for item in mentioned_users])
            users_dedup.extend(list(set(item['username'] for item in mentioned_users)))
        tweet = {
            'date': obj.get('date'),
            'id': obj.get('id'),
            'username': obj.get('user', {}).get('username'),
            'mentionedUsers': users,
            'users_dedup': users_dedup
        }
        data.append(tweet)
df = pl.DataFrame(data)

In [38]:
id_count = df.group_by('id').len()
id_count = id_count.filter(pl.col('len')>1)
len(id_count)

0

El total de registros duplicados para el dataframe por el id del tweet generado es cero para el archivo de estudio.

Para la revisión de las multiples mensiones en el mismo tweets se veficó de la siguiente manera:

In [36]:
mensiones_duplicadas = df.filter(pl.col('mentionedUsers').len() != pl.col('users_dedup').len())
len(mensiones_duplicadas)

0

### Supuestos generales

1. Los archivos para su ejecución se encuentran en la ruta: "../data/farmers-protest-tweets-2021-2-4.json"
2. Los emojis con variaciones de tono de piel serán tratados como emojis distintos.
3. Los datos que se extraen desde Twitter son unicos y no tienen duplicados.


In [1]:
file_path = "farmers-protest-tweets-2021-2-4.json"

## 3. Explicación de las funciones implementadas


### Ejercicio 1
Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.

#### Optimización de Tiempo

Para responder a este requerimiento se optó por utilizar la libreria *polars* de python, con la cual se busca cargar la información a procesar en memoria para posteriormente procesarla según las necesidades del caso.

In [ ]:
from q1_time import q1_time
result = q1_time(file_path)
print(result)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### Optimización de Memoria

Para responder a este requerimiento se optó por la lectura secuencial del archivo, por cada linea nueva que se procesa se rescatan las estadisticas necesarias para el cálculo final necesario.

In [ ]:
from q1_memory import q1_memory
result = q1_memory(file_path)
print(result)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


### Comparación de tiempo y uso de memoria para cada una de las funciones
#### Tiempo

In [ ]:
import cProfile
import pstats
from io import StringIO
from q1_time import q1_time
from q1_memory import q1_memory

output_stream = StringIO()
profiler = cProfile.Profile()
profiler.enable()
q1_time(file_path)
profiler.disable()

stats = pstats.Stats(profiler)
stats.stream = output_stream
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()

# Obtener el tottime final
tiempo_total_q1_time = stats.total_tt
tiempo_total_q1_time

output_stream = StringIO()
profiler = cProfile.Profile()
profiler.enable()
q1_memory(file_path)
profiler.disable()

stats = pstats.Stats(profiler)
stats.stream = output_stream
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()
# Obtener el tottime final
tiempo_total_q1_memory = stats.total_tt
tiempo_total_q1_memory


2.4391046240000005

In [ ]:
print(f'La funcion que prioriza el tiempo tiene un consumo de: {tiempo_total_q1_time:.4f} segundos, \nmientras que la funcion que prioriza el uso de memoria tiene un consumo de: {tiempo_total_q1_memory:.4f} segundos')
diferencia_porcentual = (abs(tiempo_total_q1_memory - tiempo_total_q1_time) / tiempo_total_q1_time) * 100
print(f'Lo que representra una mejora de {diferencia_porcentual:.4f}%')

La funcion que prioriza el tiempo tiene un consumo de: 2.4391 segundos, 
mientras que la funcion que prioriza el uso de memoria tiene un consumo de: 3.1812 segundos
Lo que representra una mejora de 30.4233%


#### Memoria

### Ejercicio 2
Los top 10 emojis más usados con su respectivo conteo.

#### Optimización de Tiempo

Para responder a este requerimiento se optó por utilizar la libreria *pandas* de python, a diferencia del resto de implementaciones se cargo todo el archivo en memoria para luego procesarlo.

`df = pd.read_json(file_path, lines=True)`

Luego utilizando la libreria *emot* se utiliza su implementación para la lectura con multiprocesador en la siguiente linea:

`emojis = emot_obj.bulk_emoji(data)`

Para luego continuar con el procesamiento y terminar contando con *Counter*.

Como mejoras a este ejercicio se propone identificar emojis con cambio de tono de piel como un solo emoji, por ejemplo: 👍🏽 y 👍, sean tratados como un solo emoji y no dos como esta en la implementación actual.


In [4]:
from q2_time import q2_time
result = q2_time(file_path)
print(result)

[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('❤', 1779), ('🤣', 1668), ('✊', 1651), ('🙏🏻', 1317), ('💚', 1040)]


#### Optimización de Memoria

Para responder a este requerimiento se optó por la lectura secuencial del archivo utilizando la libreria *jsonlines*.

`with jsonlines.open(file_path) as reader`

Por cada una de las lineas procesadas se utiliza el metodo *emoji* que retorna un diccionario con los emojis y su ubicación en el texto.

`emojis = emot_obj.emoji(content)`

Ejemplo de salida:

` {'value': ['☮', '🙂', '❤'], 'location': [[14, 15], [16, 17], [18, 19]], 'mean': [':peace_symbol:',':slightly_smiling_face:', ':red_heart:'], 'flag': True}` 

De este resultado se utilza 'value' el que se agrega al contador

`emojis = [item for item in emojis['value']]`

`contador.update(emojis)`

In [5]:
from q2_memory import q2_memory
result = q2_memory(file_path)
print(result)

[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('❤', 1779), ('🤣', 1668), ('✊', 1651), ('🙏🏻', 1317), ('💚', 1040)]


### Comparación de tiempo y uso de memoria para cada una de las funciones
#### Tiempo

In [8]:
import cProfile
import pstats
from io import StringIO
from q2_time import q2_time
from q2_memory import q2_memory

output_stream = StringIO()
profiler = cProfile.Profile()
profiler.enable()
q2_time(file_path)
profiler.disable()

stats = pstats.Stats(profiler)
stats.stream = output_stream
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()

# Obtener el tottime final
tiempo_total_q2_time = stats.total_tt

output_stream = StringIO()
profiler = cProfile.Profile()
profiler.enable()
q2_memory(file_path)
profiler.disable()

stats = pstats.Stats(profiler)
stats.stream = output_stream
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()
# Obtener el tottime final
tiempo_total_q2_memory = stats.total_tt



In [9]:
print(f'La funcion que prioriza el tiempo tiene un consumo de: {tiempo_total_q2_time:.4f} segundos, \nmientras que la funcion que prioriza el uso de memoria tiene un consumo de: {tiempo_total_q2_memory:.4f} segundos')
diferencia_porcentual = (abs(tiempo_total_q2_memory - tiempo_total_q2_time) / tiempo_total_q2_time) * 100
print(f'Lo que representra una mejora de {diferencia_porcentual:.4f}%')

La funcion que prioriza el tiempo tiene un consumo de: 10.1644 segundos, 
mientras que la funcion que prioriza el uso de memoria tiene un consumo de: 19.8816 segundos
Lo que representra una mejora de 95.6000%


#### Memoria

### Ejercicio 3
El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos

#### Optimización de Tiempo

Para responder a este requerimiento se utilizó la misma aproximación que para el ejercicio 1, en donde se lee el archivo linea por linea con *jsonlines* para de esta forma generar un consilidado de los usuarios mencionados.

```with jsonlines.open(file_path) as reader:
    for obj in reader:
        mentioned_users = obj.get('mentionedUsers', {})
        if mentioned_users:
            data.extend([item['username'] for item in mentioned_users])```

En la lista data se almacenan todos los usuarios que aparecen como menciones.
Luego se genera un dataframe de *polars* para contar, ordernar y retornar los 10 elementos más comunes.


```result = (df
            .group_by('column_0') # se agrupa por la column_0, que es el username
            .len() # Se cuentan los registros
            .sort('len',descending=True) # Se ordenan de manera descendente
            .head(10) # se retornan los 10 primeros registros
)```


In [3]:
from q1_time import q1_time
result = q1_time(file_path)
print(result)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### Optimización de Memoria

Para responder a este requerimiento se optó por la lectura secuencial del archivo, por cada linea nueva que se procesa se rescatan las estadisticas necesarias para el cálculo final necesario.

In [50]:
from q1_memory import q1_memory
result = q1_memory(file_path)
print(result)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


### Comparación de tiempo y uso de memoria para cada una de las funciones
#### Tiempo

In [51]:
import cProfile
import pstats
from io import StringIO
from q1_time import q1_time
from q1_memory import q1_memory

output_stream = StringIO()
profiler = cProfile.Profile()
profiler.enable()
q1_time(file_path)
profiler.disable()

stats = pstats.Stats(profiler)
stats.stream = output_stream
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()

# Obtener el tottime final
tiempo_total_q1_time = stats.total_tt
tiempo_total_q1_time

output_stream = StringIO()
profiler = cProfile.Profile()
profiler.enable()
q1_memory(file_path)
profiler.disable()

stats = pstats.Stats(profiler)
stats.stream = output_stream
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()
# Obtener el tottime final
tiempo_total_q1_memory = stats.total_tt
tiempo_total_q1_memory


2.4391046240000005

In [65]:
print(f'La funcion que prioriza el tiempo tiene un consumo de: {tiempo_total_q1_time:.4f} segundos, \nmientras que la funcion que prioriza el uso de memoria tiene un consumo de: {tiempo_total_q1_memory:.4f} segundos')
diferencia_porcentual = (abs(tiempo_total_q1_memory - tiempo_total_q1_time) / tiempo_total_q1_time) * 100
print(f'Lo que representra una mejora de {diferencia_porcentual:.4f}%')

La funcion que prioriza el tiempo tiene un consumo de: 2.4391 segundos, 
mientras que la funcion que prioriza el uso de memoria tiene un consumo de: 3.1812 segundos
Lo que representra una mejora de 30.4233%


#### Memoria